In [1]:
from matplotlib import pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from bokeh.io import output_notebook, show  # TODO change output
output_notebook()

Loading BokehJS ...

In [2]:
# NetworkX Fuckery
import networkx as nx

G = nx.Graph()

G.add_nodes_from((0,1,2,3,4,5))

G.add_edge(2,4)
G.add_edge(3,2)
G.add_edge(3,4)
print('vx 2 neighbors:', list(G.neighbors(2)))

G = nx.DiGraph()

G.add_nodes_from((0,1,2))
G.add_edge(0,1)
print('node 0 edges:', G.edges(0))
print('node 1 edges:', G.edges(1))
for i in G.nodes:
    print(list(G.successors(i)) == list(G.neighbors(i)))

vx 2 neighbors: [4, 3]
node 0 edges: [(0, 1)]
node 1 edges: []
True
True
True


In [3]:
# Import a gridgraph
from gridgraph.dynamic_grid import Element, Grid

# Quick unit test
crit_radius = 1
coords = [(0,0), (0,1), (0,2),
          (1,0), (1,1), (1,2),
          (2,0), (2,1), (2,2),]
mygrid = Grid(coordinates=coords,
              element_class=Element,
              crit_radius=crit_radius)

mygrid.elements[0].clear_neighbors()
mygrid.elements[0].add_neighbor(mygrid.elements[1])
mygrid.elements[0].add_neighbor(mygrid.elements[3])

print('target should be None:', mygrid.elements[0].target)

for e in mygrid.elements:
    e.target = e.neighbors[0]
print('targets:', [e.target.idx for e in mygrid.elements])
print('G edges:', mygrid.G.edges)

walk = mygrid.walk_graph(mygrid.elements[2])
print(walk)  # Here is a bottom-up traversal starting at element 2
print('check: 2, 5, 8, 1, 4, 7, 3, 6, 0')

target should be None: None
targets: [1, 2, 1, 4, 1, 2, 3, 4, 5]
G edges: [(0, 1), (1, 2), (2, 1), (3, 4), (4, 1), (5, 2), (6, 3), (7, 4), (8, 5)]
deque([2, 5, 8, 1, 4, 7, 3, 6, 0])
check: 2, 5, 8, 1, 4, 7, 3, 6, 0


In [4]:
# Display in Bokeh
import math
import numpy as np

import bokeh
from bokeh.models import Circle, StaticLayoutProvider, Plot, Range1d
from bokeh.palettes import Spectral9
from bokeh.plotting import from_networkx
from bokeh.io import output_file, reset_output
from bokeh.events import Tap

# output_file('test_bokeh.html')
#reset_output()
#output_notebook()

node_indices = list(range(len(mygrid)))
layout = mygrid.layout()

plot = Plot(x_range=Range1d(-.2, 2.2), y_range=Range1d(-.2, 2.2))
plot.background_fill_color = "midnightblue"
plot.background_fill_alpha = .15


# Mesh renderer
mesh = from_networkx(mygrid.A, layout)

N=len(mygrid)

#edge_dict = {}
#edge_dict['start'], edge_dict['end'] = mygrid.mesh()
# edge_dict['property'] = mygrid.ws   # or something
#mesh.edge_renderer.data_source.data = edge_dict
mesh.node_renderer.visible = False
mesh.edge_renderer.glyph.line_width = 1
mesh.edge_renderer.glyph.line_dash = [2, 4]
mesh.edge_renderer.glyph.line_color = 'silver'


# Graph renderer
graph = from_networkx(mygrid.G, layout)
#graph = GraphRenderer()

#edge_dict = {}
#edge_dict['start'], edge_dict['end'] = mygrid.edges()
#graph.edge_renderer.data_source.data = edge_dict
graph.node_renderer.visible = False
graph.edge_renderer.glyph.line_width = 4
graph.edge_renderer.glyph.line_color = 'hotpink'
# TODO colormap based on current, width based on wire width


# Node renderer
nodes = from_networkx(mygrid.A, layout)
nodes.edge_renderer.visible = False
nodes.node_renderer.data_source.add(node_indices, 'index')
nodes.node_renderer.data_source.add(Spectral9, 'color')
nodes.node_renderer.glyph = Circle(size=7, fill_color='lime')
# TODO special markers for the sinks


### start of layout code
#layout = mygrid.layout()
#mesh.layout_provider = StaticLayoutProvider(graph_layout=layout)
#graph.layout_provider = StaticLayoutProvider(graph_layout=layout)
#nodes.layout_provider = StaticLayoutProvider(graph_layout=layout)

plot.renderers.append(mesh)
plot.renderers.append(graph)
plot.renderers.append(nodes)

def randomize_grid():
    for e in mygrid.elements:
        e.target = np.random.choice(e.neighbors)
    graph.edge_renderer.data_source.data = mygrid.edges()

randomize_grid()

show(plot)

In [5]:
# Test change_radius
radius = 1.9
for e in mygrid.elements:
    e.target = None
graph.edge_renderer.data_source.data = {'start': [], 'end': []}

mygrid.change_radius(radius)

mesh.edge_renderer.data_source.data = mygrid.mesh()

show(plot)

In [22]:
# Test grid_generator
from gridgraph.utils import grid_generator

import numpy.random as rand

x = grid_generator(200, type='square')
x = grid_generator(20, type='rand')